**Introduction**

In [18]:
!pip install nltk 
!pip install regex
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import os
import regex as re 
import sklearn 
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("omw-1.4")
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Dealing with unstructured data, specifically text, can get tedious because it takes many forms like reviews or articles. All these forms need to be stored as documents that then need to be compiled into a corpus. In this case each document (review) is stored in a text file and a function is needed to extract the reviews from the text file and land it into a pandas dataframe. Populating a dataset with the reviews will make the corpus of text more readable and easier to analyze. Therefore, a pandas dataframe is created with two variables:

-Sentiment: the class label of the review

-Review: textual content of movie review

The unit of analysis is the sentiment expressed by the movie review text

In [23]:
#creating dataframe to land text in

#define columns
cols = ['sentiment', 'review']

#create dataframe using the columns
df = pd.DataFrame(columns = cols)

neg_path = '/workspaces/Naive-Reviews/neg/'
pos_path = '/workspaces/Naive-Reviews/pos/'

def get_docs(path, data, label):

    """ Extractor of text from text files stored in a directory. Populator of existing pandas dataframe wih extracted text data

        Args:
            path (str): path to directory containing text files
            dataframe (pd.DataFrame): dataframe to populate with extracted text
            label (int): class label of document
        Returns:
            dataframe (pd.DataFrame): dataframe with extracted text and class labels
    """
    #open directory
    folder = os.listdir(path)
    for i in folder:
        files = os.path.join(path, i)
        #extract text from each file in directory and append the values to the dataframe
        with open(files, 'r') as file:
            review_text = file.read()
            sentiment = label
            data = data.append({'sentiment': sentiment,'review': review_text}, ignore_index = True)
    return data

df = get_docs(neg_path, df, 0)
df = get_docs(pos_path, df, 1)
df.head()

AttributeError: 'DataFrame' object has no attribute 'append'

**Exploratory Data Analysis**

Preprocessing Data 

**Feature Extraction**

**Naïve Bayes Classifier Training**

**Model Testing**

**Conclusion**